In [2]:
pip install linearmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.7 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 15.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from linearmodels import RandomEffects

# 首先，读取数据
data = pd.read_csv('all_data 2.csv', encoding='ISO-8859-1')  # 替换为您数据的路径
data.head()

/tmp/ipykernel_29934/2200103434.py:5: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('all_data 2.csv', encoding='ISO-8859-1')  # 替换为您数据的路径


,host_id,id,calculated_host_listings_count,host_listings,norhost_listings,name,minimum_nights,latitude,room_type,price,...,reviews_per_monthmaxmin,nor_reviews_per,year,Unnamed: 23,Unnamed: 24,Unnamed: 25,nor_ava,nor_numb,nor_per,standard_
0,49602995,9582415.0,1,999.0,0.001001,Single/Twin/Double Ensuite near Twickenham Sta...,1,51.44473,Private room,35.0,...,118.29,0.019107,2019,NaN,0.761643836,NaN,0.761644,0.030599,0.019107,0.811787
1,70374572,23013522.0,3,0.0,0.003003,"Spacious room with double bed for 2, Twickenham",1,51.45867,Private room,19.0,...,0.01,0.055039,2019,NaN,NaN,NaN,0.063014,0.085286,0.055039,0.203576
2,25587547,25796711.0,1,NaN,0.001001,Stunning Central London Apartment Close The River,7,51.47398,Entire home/apt,103.0,...,NaN,0.004227,2019,NaN,availability_365,NaN,0.868493,0.005208,0.004227,0.879215
3,57607790,29060134.0,1,NaN,0.001001,Recently renovated fabulous four bedroom house,14,51.52716,Entire home/apt,85.0,...,NaN,0.003297,2019,NaN,max,365.0,0.947945,0.003255,0.003297,0.955559
4,228095684,30374086.0,3,NaN,0.003003,Lovely cosy flat for 4 in Heart of North London,3,51.59118,Entire home/apt,85.0,...,NaN,0.012766,2019,NaN,min,0.0,0.380822,0.010417,0.012766,0.405066


In [9]:
import statsmodels.api as sm
print(data.columns)

Index(['index', 'host_id', 'calculated_host_listings_count', 'host_listings',
       'norhost_listings', 'name', 'minimum_nights', 'latitude', 'price',
       'price_maxmin', 'nor_price', 'availability_365', 'number_of_reviews',
       'review_maxmin', 'Unnamed: 15', 'longitude', 'last_review',
       'neighbourhood', 'reviews_per_month', 'reviews_per_monthmaxmin',
       'nor_reviews_per', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'nor_ava', 'nor_numb', 'nor_per', 'standard_', 'year_2020', 'year_2021',
       'year_2022', 'year_2023', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')


In [2]:
# 将索引中的year重置为列
data = data.reset_index()
data['year_copy'] = data['year']
# 创建年份的虚拟变量
data = pd.get_dummies(data, columns=['year'], drop_first=True)

In [89]:
print(data.columns)

Index(['index', 'host_id', 'id', 'calculated_host_listings_count',
       'host_listings', 'norhost_listings', 'name', 'minimum_nights',
       'latitude', 'room_type', 'price', 'price_maxmin', 'nor_price',
       'availability_365', 'number_of_reviews', 'review_maxmin', 'Unnamed: 15',
       'longitude', 'last_review', 'neighbourhood', 'reviews_per_month',
       'reviews_per_monthmaxmin', 'nor_reviews_per', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'nor_ava', 'nor_numb', 'nor_per',
       'standard_', 'year_copy', 'year_2020', 'year_2021', 'year_2022',
       'year_2023'],
      dtype='object')


In [3]:
# 对分类变量进行独热编码
data = pd.get_dummies(data, columns=['room_type'], drop_first=True)

In [4]:
# 将id和year重新设置为索引
data = data.set_index(['id','year_copy'])

In [5]:
# 计算价格的前2%的阈值
high_price_threshold = data['price'].quantile(0.98)
low_price_threshold = data['price'].quantile(0.02)

# 移除价格在前2%之内的记录作为异常值
data_filtered = data[data['price'] < high_price_threshold]
data_filtered = data_filtered[data_filtered['price'] > low_price_threshold]

In [21]:
print(data.columns)

Index(['index', 'host_id', 'calculated_host_listings_count', 'host_listings',
       'norhost_listings', 'name', 'minimum_nights', 'latitude', 'price',
       'price_maxmin', 'nor_price', 'availability_365', 'number_of_reviews',
       'review_maxmin', 'Unnamed: 15', 'longitude', 'last_review',
       'neighbourhood', 'reviews_per_month', 'reviews_per_monthmaxmin',
       'nor_reviews_per', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'nor_ava', 'nor_numb', 'nor_per', 'standard_', 'year_2020', 'year_2021',
       'year_2022', 'year_2023', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')


In [39]:
import numpy as np
# 选择自变量和因变量

independent_vars = ['norhost_listings','nor_reviews_per','Unnamed: 15','room_type_Private room',
       'room_type_Shared room'] + [col for col in data.columns if 'year_' in col]
dependent_var = 'standard_'  # 之前计算的综合指数

# 准备模型的因变量和自变量
y = data_filtered[dependent_var]
X = data_filtered[independent_vars]

# 添加常数项
X = sm.add_constant(X)

# 构建随机效应模型
model = RandomEffects(y, X)
results = model.fit()

# 输出模型结果
print(results)

                        RandomEffects Estimation Summary                        
Dep. Variable:              standard_   R-squared:                        0.0953
Estimator:              RandomEffects   R-squared (Between):              0.0299
No. Observations:              380862   R-squared (Within):               0.0101
Date:                Tue, Dec 05 2023   R-squared (Overall):              0.0192
Time:                        01:09:56   Log-likelihood                 3.281e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4456.8
Entities:                      124834   P-value                           0.0000
Avg Obs:                       3.0509   Distribution:                F(9,380852)
Min Obs:                       1.0000                                           
Max Obs:                       342.00   F-statistic (robust):             774.51
                            

In [40]:
from linearmodels.panel import PanelOLS
exog = data_filtered[independent_vars]
# 确保 'data' 是多级索引的 DataFrame，索引为 'id' 和 'year'
mod = PanelOLS(y, exog)
res = mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:              standard_   R-squared:                        0.4236
Estimator:                   PanelOLS   R-squared (Between):              0.4608
No. Observations:              380862   R-squared (Within):              -0.2295
Date:                Tue, Dec 05 2023   R-squared (Overall):              0.4236
Time:                        01:10:04   Log-likelihood                -1.788e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   3.111e+04
Entities:                      124834   P-value                           0.0000
Avg Obs:                       3.0509   Distribution:                F(9,380853)
Min Obs:                       1.0000                                           
Max Obs:                       342.00   F-statistic (robust):          3.111e+04
                            

In [35]:
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
import statsmodels.api as sm
from scipy import stats

# 固定效应模型
fe_model = PanelOLS(data_filtered[dependent_var], data_filtered[independent_vars], entity_effects=True)
fe_result = fe_model.fit()

# 随机效应模型
re_model = RandomEffects(data_filtered[dependent_var], data_filtered[independent_vars])
re_result = re_model.fit()

# 豪斯曼检验
b = fe_result.params
B = re_result.params
v_b = fe_result.cov
v_B = re_result.cov

diff = b - B
var_diff = np.diag(v_b) + np.diag(v_B) - 2 * np.diag(v_b @ v_B)
dof = len(diff)
chi2 = np.sum(diff**2 / var_diff)
pval = stats.chi2.sf(chi2, dof)

print('Hausman Test')
print(f'chi2 statistic: {chi2}')
print(f'p-value: {pval}')

Hausman Test
chi2 statistic: 21839.247460184637
p-value: 0.0


In [36]:
print(fe_result)

                          PanelOLS Estimation Summary                           
Dep. Variable:              standard_   R-squared:                        0.0137
Estimator:                   PanelOLS   R-squared (Between):              0.0299
No. Observations:              380862   R-squared (Within):               0.0137
Date:                Tue, Dec 05 2023   R-squared (Overall):              0.0029
Time:                        00:56:43   Log-likelihood                 1.098e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      395.38
Entities:                      124834   P-value                           0.0000
Avg Obs:                       3.0509   Distribution:                F(9,256019)
Min Obs:                       1.0000                                           
Max Obs:                       342.00   F-statistic (robust):             395.38
                            